In [2]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold
import torch

In [3]:

if torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set to your desired GPU number
else:
    print("No CUDA GPUs are available.")

In [3]:
#label extraction
dataset_path = Path('datasets\\yolo') 
labels = sorted(dataset_path.rglob("*labels/*.txt")) 

In [11]:


classes = ['Car','Cyclist','Pedestrian','Truck','Van','Tram','Misc','Person_sitting','DontCare']
cls_idx = [0,1,2,3,4,5,6,7,8]
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)


In [12]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

C:\Users\James\AppData\Local\Temp\ipykernel_8316\2598033761.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`


In [13]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [14]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

C:\Users\James\AppData\Local\Temp\ipykernel_8316\3825890032.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
C:\Users\James\AppData\Local\Temp\ipykernel_8316\3825890032.p

In [15]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [16]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

datasets\yolo\2024-04-08_5-Fold_Cross-val


In [18]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        print(split, k_split)
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        # shutil.copy(image, img_to_path / image.name)
        # shutil.copy(label, lbl_to_path / label.name)

split_1 val
split_2 train
split_3 train
split_4 train
split_5 train
split_1 train
split_2 train
split_3 val
split_4 train
split_5 train
split_1 train
split_2 train
split_3 train
split_4 train
split_5 val
split_1 train
split_2 train
split_3 train
split_4 train
split_5 val
split_1 train
split_2 train
split_3 train
split_4 val
split_5 train
split_1 train
split_2 val
split_3 train
split_4 train
split_5 train
split_1 train
split_2 val
split_3 train
split_4 train
split_5 train
split_1 train
split_2 train
split_3 val
split_4 train
split_5 train
split_1 train
split_2 train
split_3 train
split_4 val
split_5 train
split_1 val
split_2 train
split_3 train
split_4 train
split_5 train
split_1 train
split_2 train
split_3 train
split_4 val
split_5 train
split_1 train
split_2 train
split_3 val
split_4 train
split_5 train
split_1 val
split_2 train
split_3 train
split_4 train
split_5 train
split_1 train
split_2 train
split_3 val
split_4 train
split_5 train
split_1 val
split_2 train
split_3 train
split_4 

In [4]:

weights_path = 'yolov8n.pt' 
# pretrained model can be changed to yolov8n.pt, for faster training

model = YOLO(weights_path, task='detect')
results = {}

project = 'kfold_demo'
epochs = 100
patience= 3

for k in range(5):
    dataset_yaml = f'datasets\yolo\{datetime.date.today().isoformat()}_5-Fold_Cross-val\split_{k+1}\split_{k+1}_dataset.yaml'
    model.train(data=dataset_yaml,epochs=epochs, project=project, device="gpu", patience=patience)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

New https://pypi.org/project/ultralytics/8.1.45 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42  Python-3.11.3 torch-2.2.2+cu121 CUDA:gpu (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets\yolo\2024-04-08_5-Fold_Cross-val\split_1\split_1_dataset.yaml, epochs=100, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=kfold_demo, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fa

train: Scanning C:\Users\James\Desktop\MacOS\ML\datasets\yolo\2024-04-08_5-Fold_Cross-val\split_1\train\labels.cache... 6339 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6339/6339 [00:00<?, ?it/s]
val: Scanning C:\Users\James\Desktop\MacOS\ML\datasets\yolo\2024-04-08_5-Fold_Cross-val\split_1\val\labels.cache... 1585 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1585/1585 [00:00<?, ?it/s]


Plotting labels to kfold_demo\train22\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to kfold_demo\train22
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.35G      1.679      2.022      1.169         21        640: 100%|██████████| 397/397 [00:49<00:00,  7.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:09<00:00,  5.47it/s]


                   all       1585      12923      0.519      0.437      0.451      0.265

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1585      12923      0.932      0.879      0.936      0.735
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 87, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



90 epochs completed in 1.223 hours.
Optimizer stripped from kfold_demo\train22\weights\last.pt, 6.2MB
Optimizer stripped from kfold_demo\train22\weights\best.pt, 6.2MB

Validating kfold_demo\train22\weights\best.pt...
Ultralytics YOLOv8.1.42  Python-3.11.3 torch-2.2.2+cu121 CUDA:gpu (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:10<00:00,  4.92it/s]


                   all       1585      12923      0.931      0.877      0.935      0.735
                   Car       1585       5449      0.937      0.932      0.973      0.822
               Cyclist       1585        366      0.959      0.891      0.955      0.748
            Pedestrian       1585       2192      0.928      0.803        0.9      0.607
                 Truck       1585        246      0.975      0.965       0.99      0.903
                   Van       1585        683      0.969      0.937      0.983      0.806
                  Tram       1585         94      0.938      0.968      0.979      0.849
                  Misc       1585        170      0.938      0.876      0.924      0.742
        Person_sitting       1585        125      0.868      0.856       0.92      0.716
              DontCare       1585       3598      0.869      0.665      0.791      0.419
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to kfold_demo\

train: Scanning C:\Users\James\Desktop\MacOS\ML\datasets\yolo\2024-04-08_5-Fold_Cross-val\split_2\train\labels.cache... 6339 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6339/6339 [00:00<?, ?it/s]
val: Scanning C:\Users\James\Desktop\MacOS\ML\datasets\yolo\2024-04-08_5-Fold_Cross-val\split_2\val\labels.cache... 1585 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1585/1585 [00:00<?, ?it/s]
